In [1]:
import pandas as pd
import numpy as np
import flow

In [2]:
df = flow.get_electricity_generation_data()
df.head()

,Plant Id,Combined Heat And\nPower Plant,Nuclear Unit Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,Reserved,...,Netgen\nSeptember,Netgen\nOctober,Netgen\nNovember,Netgen\nDecember,Total Fuel Consumption\nQuantity,Electric Fuel Consumption\nQuantity,Total Fuel Consumption\nMMBtu,Elec Fuel Consumption\nMMBtu,Net Generation\n(Megawatthours),YEAR
0,2,N,NaN,Bankhead Dam,Alabama Power Co,195,AL,ESC,SERC,NaN,...,974,"1,729","2,698","3,881",0,0,"241,548","241,548","25,920",2015
1,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,"126,735","145,359","244,112","223,707","770,956","770,956","788,125","788,125","2,464,536",2015
2,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,"217,458","254,183","441,658","424,910","46,149,030","46,149,030","47,195,389","47,195,389","4,318,717",2015
3,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,"495,747","337,972","308,318","352,781","2,153,948","2,153,948","46,083,543","46,083,543","4,559,961",2015
4,3,N,NaN,Barry,Alabama Power Co,195,AL,ESC,SERC,NaN,...,"1,351","1,970","3,595","1,184","446,649","446,649","457,069","457,069","44,348",2015
